This is a modified version of the code published by TensorFlow https://www.tensorflow.org/alpha/tutorials/text/nmt_with_attention

##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License").

# Neural Machine Translation with Attention


In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.13.1


In [3]:
#download the dataset
!wget http://www.manythings.org/anki/ara-eng.zip
!unzip ara-eng.zip

--2019-05-30 19:16:01--  http://www.manythings.org/anki/ara-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264050 (258K) [application/zip]
Saving to: ‘ara-eng.zip’

ara-eng.zip         100%[===================>] 257.86K  --.-KB/s    in 0.1s    

2019-05-30 19:16:02 (2.23 MB/s) - ‘ara-eng.zip’ saved [264050/264050]

Archive:  ara-eng.zip
  inflating: _about.txt              
  inflating: ara.txt                 


In [2]:
!cat ara.txt | head -5

Hi.	مرحبًا.
Run!	اركض!
Help!	النجدة!
Jump!	اقفز!
Stop!	قف!


In [0]:
path_to_file = "ara.txt"

## Prepare the dataset

Each line in the dataset contains a line with two statements one in Arabic and one in English

`Call me.	اتصل بي.`

After downloading the dataset, here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    #w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [0]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    print(lines[0].split('\t'))
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return word_pairs

In [0]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)

    # index language using the class defined above    
    inp_lang = LanguageIndex(sp for en, sp in pairs)
    targ_lang = LanguageIndex(en for en, sp in pairs)
    
    # Vectorize the input and target languages
    
    # Spanish sentences
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    
    # English sentences
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

### Limit the size of the dataset to experiment faster (optional)

You can reduce the num_examples to train faster

In [9]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

['Hi.', 'مرحبًا.']


In [10]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.01)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(10830, 10830, 110, 110)

### Create a tf.data dataset

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 256
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

## Write the encoder and decoder model

Here, we'll implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input word is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence.

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*. 

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

We're using *Bahdanau attention*. Lets decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, 1)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU
  
The shapes of all the vectors at each step have been specified in the comments in the code:

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  return tf.keras.layers.GRU(units, 
                             return_sequences=True, 
                             return_state=True, 
                             recurrent_activation='sigmoid', 
                             recurrent_initializer='glorot_uniform')

In [0]:
def get_encoder(vocab_size, embedding_dim, enc_units, batch_sz):
    hidden = tf.zeros((batch_sz, enc_units))
    input = tf.keras.layers.Input((max_length_inp,))
    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(input)
    x = gru(units)(x)
    return tf.keras.models.Model(inputs = input, outputs = x)

In [0]:
def get_decoder(vocab_size, embedding_dim, units, batch_sz):
  
    #define the inputs to the decoder
    enc_output = tf.keras.layers.Input((max_length_inp, embedding_dim))
    enc_hidden = tf.keras.layers.Input((embedding_dim,))
    dec_input = tf.keras.layers.Input((1,))
    
    hidden_with_time_axis = tf.keras.layers.Reshape((1, embedding_dim))(enc_hidden)

    # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
    W1 = tf.keras.layers.Dense(units)
    W2 = tf.keras.layers.Dense(units)
    V  = tf.keras.layers.Dense(1, activation = 'softmax')
    
    attention_weights = V(tf.keras.layers.Activation(activation = "tanh")(tf.keras.layers.Add()([W1(enc_output), W2(hidden_with_time_axis)])))
    
    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = tf.keras.layers.Multiply()([attention_weights, enc_output])
    
    # reshape the context_vector to concatneate with the output of the first input
    context_vector = tf.keras.layers.Permute((2, 1))(context_vector)
    context_vector = tf.keras.layers.Dense(1)(context_vector)
    context_vector = tf.keras.layers.Permute((2, 1))(context_vector)
    
    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(dec_input)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.keras.layers.Concatenate(axis = -1)([context_vector, x])

    # passing the concatenated vector to the GRU
    output, state = gru(units)(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.keras.layers.Reshape((output.shape[2],))(output)

    # output shape == (batch_size * 1, vocab)
    x = tf.keras.layers.Dense(vocab_size)(output)

    return tf.keras.models.Model(inputs = [dec_input, enc_hidden, enc_output], outputs = x)

In [15]:
encoder = get_encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = get_decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


## Define the optimizer and the loss function

In [0]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [18]:
EPOCHS = 30

for epoch in range(EPOCHS):
    start = time.time()
    
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp)
            #print(tf.reduce_sum(enc_output))
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                #print(dec_input.shape, dec_hidden.shape, enc_output.shape)
                predictions = decoder([dec_input, dec_hidden, enc_output])
                loss += loss_function(targ[:, t], predictions)
                
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1 Batch 0 Loss 1.5737
Epoch 1 Batch 100 Loss 0.9952
Epoch 1 Loss 1.0801
Time taken for 1 epoch 184.59089255332947 sec

Epoch 2 Batch 0 Loss 0.9520
Epoch 2 Batch 100 Loss 0.9073
Epoch 2 Loss 0.9059
Time taken for 1 epoch 185.6432590484619 sec

Epoch 3 Batch 0 Loss 0.8685
Epoch 3 Batch 100 Loss 0.8189
Epoch 3 Loss 0.8228
Time taken for 1 epoch 184.95331001281738 sec

Epoch 4 Batch 0 Loss 0.7762
Epoch 4 Batch 100 Loss 0.7495
Epoch 4 Loss 0.7631
Time taken for 1 epoch 184.47491669654846 sec

Epoch 5 Batch 0 Loss 0.7334
Epoch 5 Batch 100 Loss 0.7252
Epoch 5 Loss 0.7156
Time taken for 1 epoch 183.40274953842163 sec

Epoch 6 Batch 0 Loss 0.6917
Epoch 6 Batch 100 Loss 0.7698
Epoch 6 Loss 0.6743
Time taken for 1 epoch 185.21108555793762 sec

Epoch 7 Batch 0 Loss 0.6076
Epoch 7 Batch 100 Loss 0.7121
Epoch 7 Loss 0.6360
Time taken for 1 epoch 183.29096484184265 sec

Epoch 8 Batch 0 Loss 0.6804
Epoch 8 Batch 100 Lo

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [35]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''

    enc_out, enc_hidden = encoder(inputs)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)
    
    for t in range(max_length_targ):
        
        
        predictions = decoder([dec_input, dec_hidden, enc_out])
        

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [70]:
translate(u'اين انت ؟', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> اين انت ؟ <end>
Predicted translation: where are you ? <end> 


In [71]:
translate(u'من أين أنت', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> من اين انت <end>
Predicted translation: where are you going ? <end> 


In [72]:
# wrong translation
translate(u'اهلا', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> اهلا <end>
Predicted translation: don't worry . <end> 


In [73]:
translate(u'اذهب إلى المدرسة', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> اذهب الى المدرسة <end>
Predicted translation: i go to school . <end> 


In [74]:
translate(u'اصمت', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> اصمت <end>
Predicted translation: shut up ! <end> 


In [75]:
translate(u'اخرج من هنا', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> اخرج من هنا <end>
Predicted translation: get out of here ? <end> 


In [76]:
translate(u'هل ستذهب الى المسجد', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> هل ستذهب الى المسجد <end>
Predicted translation: will you go to america ? <end> 


In [77]:
translate(u'الهزيمة كانت قاسية', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> الهزيمة كانت قاسية <end>
Predicted translation: what was your car ? <end> 


In [78]:
translate(u'اركض', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

Input: <start> اركض <end>
Predicted translation: i turned down . <end> 


##  Save the models and the dictionaries

In [0]:
encoder.save('encoder.h5')
decoder.save('decoder.h5')

In [0]:
import csv
def create_csv(file, dict):
    with open(file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for key in dict.keys():
            writer.writerow([key,dict[key]])

In [0]:
create_csv('idx2word.csv', targ_lang.idx2word)
create_csv('word2idx.csv', inp_lang.word2idx)